#### De parama padi da

In [1]:
# all parameters we need
import numpy as np
R = 8.31446261815324

In [4]:
p = 0.5 #Mpa
pc = np.array([4.600155, 1.296960, 22.048320]) #Mpa
tc = np.array([190.6, 33.2, 647.3])
w = np.array([0.011,-0.22, 0.344])
bip = np.array([[0,	0.202000007,	0.5],
                [0.202000007,	0,	-0.299800009],
                [0.5,	-0.299800009,	0]])

comp = np.array([0.3,0.1,0.6])

ki = 0.37464 + 1.54226*w - 0.26992*w**2

### Now combine all

In [13]:
def findPhi(t, y, isV = True):
    alphai = (1 + ki * (1 - (t/tc)**0.5))**2
    ai = 0.4572355289 * (R * tc)**2 * alphai / pc
    bi = 0.0777960739 * R * tc / pc

    a = 0
    b = 0
    aik = np.zeros(y.size)

    for i in range(0,y.size):
        for j in range(0, y.size):
            aij = ((ai[i] * ai[j])**0.5 * (1 - bip[i][j]))
            axij = aij * y[i] * y[j]
            a = a + axij
        aik[i] = aik[i] + aij * y[i]

    for i in range(0,y.size):
        b = b + y[i]*bi[i]
    
    A = a*p/(R*t)**2
    B = (b*p)/(R*t)
    C2 = (B-1)
    C1 = (A - 3*B**2 - 2*B)
    C0 = (B**3 + B**2 - A*B)

    coeff = np.array([1,C2,C1,C0])
    z = np.array(np.roots(coeff))
    
    if(np.isreal(z[0])):
        zv = np.real(z[0])
    else:
        zv = 1
    
    if(np.isreal(z[2])):
        zl = np.real(z[2])
    else:
        zl = 1

    # print(zv, zl)
    
    temp1 = bi / b
    phi = np.ones(y.size)
    if(isV):
        phi = np.exp(((temp1) * (zv - 1)) -  (np.log(zv - B)) - (A / (2*np.sqrt(2)*B)) *  ((2 * aik / a) - temp1) * np.log((zv + 2.414213562373095*B) / (zv - 0.41421356237309515*B)))
    else:
        phi = np.exp(((temp1) * (zl - 1)) -  (np.log(zl - B)) - (A / (2*np.sqrt(2)*B)) *  ((2 * aik / a) - temp1) * np.log((zl + 2.414213562373095*B) / (zl - 0.41421356237309515*B)))
    
    return phi


In [14]:
def dewptfunc(t):
    phiV = findPhi(t, comp)
    # print("phiV : ", phiV)

    x_guess = comp / np.exp(np.log((pc) / p) + (5.37269855031944 * (1 + w) * (1 - (tc / t))))
    x_guess_total_old = np.sum(x_guess)
    x = x_guess / x_guess_total_old
    # print("First X : ",x)

    for i in range(0,20):
        phiL = findPhi(t, x, False)
        x_guess = comp * phiV / phiL
        # print("PhiL : ", phiL)
        # print("x_guess : ",x_guess)
        x_guess_total_new = np.sum(x_guess)
        x = x_guess / x_guess_total_new
        # print("x : ",x)
        if(abs(x_guess_total_new - x_guess_total_old)<=0.0001):
            break
        x_guess_total_old = x_guess_total_new

    return x_guess_total_old - 1


In [15]:
print(dewptfunc(133+273.15))

0.03923346523524085


In [16]:
from scipy.optimize import root_scalar as sec
ans = sec(dewptfunc, method='secant', xtol=1e-4, maxiter=50, x0=100+273.15, x1=150+273.15)

In [19]:
ans.root - 273.15

134.31531315105997